# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else... :/

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾'],
['🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🐊', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌲', '🌲', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🪨', '🪨', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🌾', '🌾', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🪨', '🪨', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🪨', '🪨', '🗻', '🗻', '🌾', '🗻', '🗻', '🪨', '🪨', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾']
]

## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [2]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [3]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [4]:
COSTS = { '🌾': 1, '🌲': 3, '🪨': 5, '🐊': 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [5]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="heuristic"></a>
## heuristic

`heuristic` estimates the cost from the current node to goal node called h(x). 
This function use the Manhattan Distance to estimate the cost. This assumes the cheapest
cost to the goal (cost of one per move). 

* **current_pos** Tuple[int, int]: current position (col, row).
* **goal_pos**: Tuple[int, int], goal position (col, row).


**returns** int: hx.

In [6]:
def heuristic(current_pos: Tuple[int, int], goal_pos: Tuple[int, int]) -> int:
    hx = abs(current_pos[1] - goal_pos[1]) + abs(current_pos[0] - goal_pos[0])
    return hx

In [7]:
# assertions/unit tests
hx1 = heuristic((0,0),(5,5))
assert hx1 == 10

hx2 = heuristic((10,10),(5,5))
assert hx2 == 10

hx2 = heuristic((3,1),(0,0))
assert hx2 == 4

<a id="getChildren"></a>
## getChildren

`getChildren` returns a list of successor nodes and the current cost for that move. 
- This is a list of legal one moves from the current position. 

* **wolrd** List[List[str]]: the map of where to travel within
* **parent** dict: the parent/current state
* **moves** List[Tuple[int, int]]: contains the legal moves allowed
* **costs** Dict[str, int]: mapping of the cost per type of spaces landed on
* **fx** callable: callable heuristic function for calculating the cost
* **goal** tuple: the goal position

**returns** list[dict]: list of children states

In [8]:
def getChildren( world: List[List[str]], parent:dict,moves: List[Tuple[int, int]], costs: Dict[str, int], fx:Callable, goal:tuple)-> list[dict]:
    kids = []

    for mv in moves:
        c_x, c_y = parent["loc"][0] +mv[0], parent["loc"][1]+ mv[1] # child pos
        new_loc = c_x, c_y
        if 0 <= c_x < len(world[0]) and 0 <= c_y < len(world)\
            and world[c_y][c_x] != '🗻': # if a valid move
            gn = parent["gn"] + costs[world[c_y][c_x]]
            cost = gn + fx(new_loc, goal)
            
            # create child
            child = {"loc": new_loc, "move": mv,"cost":cost, "gn":gn}
            child_copy = deepcopy(child)
            path = deepcopy(parent["path"])
            path.append(child_copy) # add child to parent path
            child["path"] = path
            kids.append(child)
    return kids 

In [9]:
# check at starting point with empty explored list
parent_test = {'loc': (0, 0), 'move': None, 'cost': 12, 'gn': 0, 'path': [{'loc': (0, 0), 'move': None, 'cost': 12}]}
g = (len(small_world[0])-1, len(small_world)-1)
first_moves =getChildren( small_world, parent_test,MOVES, COSTS, heuristic, g)
assert len(first_moves) == 2
assert first_moves[0]["loc"] == (1,0)
assert first_moves[1]["loc"] == (0,1)
assert len(first_moves[0]["path"])  == 2

# check bottom right
parent_test = {'loc': (6, 6), 'move': None, 'cost': 12, 'gn': 0, 'path': [{'loc': (0, 0), 'move': None, 'cost': 12}]}
g = (len(small_world[0])-1, len(small_world)-1)
bottom_moves = getChildren( small_world, parent_test,MOVES, COSTS, heuristic, g)
assert len(bottom_moves) == 2
assert bottom_moves[0]["loc"] == (6,5)
assert bottom_moves[1]["loc"] == (5,6)
assert len(bottom_moves[0]["path"])  == 2

#middle of map 
parent_test = {'loc': (4, 3), 'move': None, 'cost': 12, 'gn': 0, 'path': [{'loc': (0, 0), 'move': None, 'cost': 12}]}
g = (len(small_world[0])-1, len(small_world)-1)
middle_moves = getChildren( small_world, parent_test,MOVES, COSTS, heuristic, g)
assert len(middle_moves) == 4
assert middle_moves[0]["loc"] == (4,2)
assert middle_moves[1]["loc"] == (5,3)
assert middle_moves[2]["loc"] == (4,4)
assert middle_moves[3]["loc"] == (3,3)
assert len(middle_moves[0]["path"])  == 2

<a id="clean_frontier"></a>
## clean_frontier

- removes the first element on the frontier
- LIFO order
- Then returns that removed state and the updated/cleaned frontier

* **current_frontier** list[dict]: dictionary of locations to be explored with its f(n) value


**returns** Tuple(dict, list[dict]): returns new state to explore and the updated frontier

In [10]:
def clean_frontier(current_frontier:list[dict])->Tuple[dict, list[dict]]:
    if len(current_frontier) ==  0:
        print("Frontier is empty. Error")
        return None, []
    result = current_frontier.pop(0) # take off first element on frontier
    return result, current_frontier

In [11]:

test_frontier = [{'loc': (0, 0), 'move': None, 'cost': 12, 'gn': 0}, {'loc': (1, 0), 'move': None, 'cost': 12, 'gn': 0}
                 , {'loc': (2, 0), 'move': None, 'cost': 12, 'gn': 0}]

test_state, nfront = clean_frontier(test_frontier)
# verify first element is returned
assert test_state == {'loc': (0, 0), 'move': None, 'cost': 12, 'gn': 0}

# verify frontier is less than one now
assert len(nfront) == 2

# verify empty frontier retuns None and empty frontier
empty_state, empty_front = clean_frontier([])
assert empty_state == None
assert empty_front == []


Frontier is empty. Error


<a id="add_to_frontier"></a>
## add_to_frontier

- Adds a state to the frontier and ensure the frontier is in order based on cost
- Cheaper costs on at the beginning of the list (index 0)

* **frontier** list[dict]: dictionary of locations to be explored with its f(n) value
* **state** dict: state to add to the frontier


**returns** list[dict]: frontier with added state if applicable

In [12]:
def add_to_frontier(frontier:list[dict], state:dict)->list[dict]:
    # check if already on frontier
    # Max size should be 1
    deja_vu = [i for i in range(len(frontier)) if frontier[i]["loc"] == state["loc"]]

    # if not in frontier or its a cheaper cost
    if len(deja_vu) == 0 or (len(deja_vu) > 0  
                           and state["cost"] < frontier[deja_vu[0]]["cost"]):
        if len(deja_vu) > 0 : # if second condition
            del frontier[deja_vu[0]]
        frontier.append(state)
        # keep frontier sorted. Lower cost at beginning of frontier
        frontier.sort(key=lambda s:s["cost"]) 
    return frontier

In [13]:
# verify an element gets added to the empty frontier
test_state = {'loc': (2, 0), 'move': None, 'cost': 12, 'gn': 0}
frontier_1 = add_to_frontier([], test_state)
assert frontier_1 == [test_state]

# verify value is not added is already on the frontier and cost is more expensive
test_frontier = [{'loc': (0, 0), 'move': None, 'cost': 12, 'gn': 0}, {'loc': (1, 0), 'move': None, 'cost': 12, 'gn': 0}
                 , {'loc': (2, 0), 'move': None, 'cost': 12, 'gn': 0}]
test_frontier_copy = deepcopy(test_frontier)
test_state2 = {'loc': (2, 0), 'move': None, 'cost': 15, 'gn': 0}
frontier_2 = add_to_frontier(test_frontier, test_state2)
assert frontier_2 == test_frontier_copy

#verify the value is added is on frontier and cheaper
test_state3 = {'loc': (2, 0), 'move': None, 'cost': 8, 'gn': 0}
frontier_3 = add_to_frontier(test_frontier, test_state3)
assert frontier_3 ==  [{'loc': (2, 0), 'move': None, 'cost': 8, 'gn': 0},
{'loc': (0, 0), 'move': None, 'cost': 12, 'gn': 0}, {'loc': (1, 0), 'move': None, 'cost': 12, 'gn': 0}]


<a id="get_all_moves"></a>
## get_all_moves

- Given a path, returns only the moves. 

* **path** list[dict]: path of start to goal


**returns** list[tuple]: returns all the moves (x,y) moves

In [14]:
def get_all_moves(path:list[dict])->list[tuple]:
    moves = []
    for mv in path[1:]: # first move is None
        moves.append(mv["move"])
    return moves


In [15]:
test_path =[{'loc': (0, 0), 'move': None, 'cost': 12},
 {'loc': (0, 1), 'move': (0, 1), 'cost': 12, 'gn': 1},
 {'loc': (0, 2), 'move': (0, 1), 'cost': 12, 'gn': 2},
 {'loc': (0, 3), 'move': (0, 1), 'cost': 12, 'gn': 3},
 {'loc': (1, 3), 'move': (1, 0), 'cost': 12, 'gn': 4},
 {'loc': (2, 3), 'move': (1, 0), 'cost': 12, 'gn': 5},
 {'loc': (3, 3), 'move': (1, 0), 'cost': 12, 'gn': 6},
 {'loc': (4, 3), 'move': (1, 0), 'cost': 12, 'gn': 7},
 {'loc': (5, 3), 'move': (1, 0), 'cost': 12, 'gn': 8},
 {'loc': (6, 3), 'move': (1, 0), 'cost': 12, 'gn': 9},
 {'loc': (6, 4), 'move': (0, 1), 'cost': 12, 'gn': 10},
 {'loc': (6, 5), 'move': (0, 1), 'cost': 12, 'gn': 11},
 {'loc': (6, 6), 'move': (0, 1), 'cost': 12, 'gn': 12}]

test_mvs = get_all_moves(test_path)
# verify the length is the same -1
assert len(test_mvs) == len(test_path) -1
assert test_mvs ==[(0, 1),(0, 1),(0, 1),(1, 0),(1, 0),(1, 0),(1, 0),(1, 0),(1, 0),(0, 1),(0, 1),(0, 1)]

# verify can handle a path where start is the goal
start_goal = [{'loc': (0, 0), 'move': None, 'cost': 12}]
start_goal_moves =  get_all_moves(start_goal)
assert start_goal_moves == []

<a id="a_star_search"></a>
## a_star_search

This function returns the path from the start psotion to the goal within the world.
This is done by building a list of explored paths then finding the true path from
the explored path.

The explored path is built from finding paths with the lowest F(N)
this H(N) + g(N) 
H(N) = greedy search, estimated cost from current position to goal
G(N) = the cost so far from the explored path

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [16]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    cost = heuristic(start, goal)
    start_path = [{"loc":start, "move":None, "cost":cost}]
    start_pos = {"loc":start, "move":None, "cost":cost, "gn":0,"path": start_path}
    frontier = add_to_frontier([], start_pos)
    explored = []
    while len(frontier) > 0:
        current_state, frontier = clean_frontier( frontier)
   
        if current_state["loc"] == goal:
            return get_all_moves(current_state["path"]) 
        children = getChildren(world, current_state, moves, costs, heuristic, goal)

        for child in children:
            if not len([ state for state in explored if state["loc"] == child["loc"]]) > 0:
                frontier = add_to_frontier(frontier, child)
        explored.append(current_state)
    return None

In [17]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
s2 = [(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]
assert small_path == s2
#check path is list of actions rather than locations
for one_move in small_path:
    assert one_move in  MOVES
#check if start == goal
small_path = a_star_search(small_world, (5,2), (5,2), COSTS, MOVES, heuristic)
assert len(small_path) == 0
# check if goal right next to start. Len should be one
small_path = a_star_search(small_world, (5,2), (5,1), COSTS, MOVES, heuristic)
assert len(small_path) == 1
assert small_path == [(0,-1)] # should be move up only

## pretty_print_path

`pretty_print_path` will create a new map with the actions taken to get to the
goal. This map will be printer. While the map is being created, the cost for each 
move will be stored in a variable, which is returned

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [18]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    actions = {(1,0) : '⏩', (-1,0) :'⏪',(0,1):'⏬', (0,-1):'⏫'}
    total_costs = 0  # cost all actions
    travel = start # pointer of position
    new_world = deepcopy(world)
    new_world[goal[1]][goal[0]] = '🎁'

    for move in path: # update map
        total_costs += costs[world[travel[1]][travel[0]]] # add cost
        new_world[travel[1]][travel[0]] = actions[move] # update map
        travel_col = travel[0] + move[0]
        travel_row = travel[1] + move[1]
        travel = (travel_col, travel_row)

    for rw in new_world: # print map
        for i in rw:
            print(i, end=' ')
        print()

    return total_costs # return total costs

In [19]:
dup_map = [['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
 ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
 ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
 ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
 ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
 ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
 ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']]
small_start = (0, 2)
small_path = [(0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0)]
small_goal = (6,3)
c = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
# assert returns correct cost
assert c == 7
# check small world doesn't change
assert small_world == dup_map

print("Next map")
small_start = (0, 6)
small_path = [(1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,-1)]
small_goal = (6,5)
c = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
assert c == 19

🌾 🌲 🌲 🌲 🌲 🌲 🌲 
🌾 🌲 🌲 🌲 🌲 🌲 🌲 
⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏩ ⏩ ⏩ ⏩ ⏩ ⏩ 🎁 
🌲 🌲 🌲 🌲 🌲 🌲 🌾 
🌲 🌲 🌲 🌲 🌲 🌲 🌾 
🌲 🌲 🌲 🌲 🌲 🌲 🌾 
Next map
🌾 🌲 🌲 🌲 🌲 🌲 🌲 
🌾 🌲 🌲 🌲 🌲 🌲 🌲 
🌾 🌲 🌲 🌲 🌲 🌲 🌲 
🌾 🌾 🌾 🌾 🌾 🌾 🌾 
🌲 🌲 🌲 🌲 🌲 🌲 🌾 
🌲 🌲 🌲 🌲 🌲 🌲 🎁 
⏩ ⏩ ⏩ ⏩ ⏩ ⏩ ⏫ 


## Problems

## Problem 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [20]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏩ ⏩ ⏩ ⏩ ⏩ ⏩ ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 🎁 
total path cost: 12
[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [21]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬ 🌾 🌾 🌾 🌾 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 
⏬ 🌾 🌾 🌾 🌾 🌾 🌾 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🗻 🗻 🗻 🗻 🗻 🗻 🗻 🌾 🌾 
⏬ 🌾 🌾 🌾 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🪨 🪨 🪨 🗻 🗻 🪨 🪨 
⏬ 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🌲 🌲 🌲 🌲 🐊 🐊 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🪨 🪨 🗻 🗻 🪨 🌾 
⏬ 🌾 🌾 🪨 🪨 🗻 🗻 🌲 🌲 🌾 🌾 🐊 🐊 🐊 🐊 🌲 🌲 🌲 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🪨 🌾 
⏬ 🪨 🪨 🪨 🗻 🗻 🪨 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🐊 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🪨 🌾 🌾 
⏬ 🪨 🪨 🗻 🗻 🪨 🪨 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🐊 🐊 🐊 🌾 🌾 🌾 🌾 🌾 🪨 🌾 🌾 🌾 
⏬ 🌾 🪨 🪨 🪨 🪨 🪨 🌾 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🌾 🌾 
⏬ 🌾 🌾 🪨 🪨 🪨 🌾 🌾 🌾 🌾 🌾 🌾 🪨 🪨 🗻 🗻 🌾 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🌾 🌾 🌾 
⏬ 🌾 🌾 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🗻 🗻 🗻 🗻 🌾 🌾 🌾 🐊 🌾 🪨 🪨 🪨 🌾 🌾 🌾 🌾 
⏬ 🌾 🐊 🐊 🐊 🐊 🐊 🌾 🪨 🪨 🗻 🗻 🗻 🪨 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🪨 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🌾 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🗻 🗻 🪨 🌾 🐊 🐊 🐊 🐊 🌾 🌾 🌾 🪨 🗻 🪨 🌾 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🌲 🌲 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🌾 🌾 🌾 🌾 🌾 
⏩ ⏬ 🌾 🌾 🗻 🌾 🌾 🌲 🌲 🌲 🌲 🪨 🪨 🪨 🪨 🌾 🐊 🐊 🐊 🌾 🌾 🪨 🗻 🪨 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🪨 🪨 🌾 🐊 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 
🌾 ⏬ 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🌾 🌾 🗻 🗻 🗻 🌾 🌾 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🗻 🌾 🌾 🌾 🌾 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 

## Comments
- A little bit of a combo of using my original code submitted plus help from the mod 1 walk through 
- https://wse.zoom.us/rec/play/1PSeZm0YJcloBJISBBeraeWYsSuBuKOvceZhl-87osAAzGhpaRhec5ohBnzT7OvR-Nk6wl7saZxz1Jk_.bc4lAxBkDsVAr5u_?canPlayFromShare=true&from=share_recording_detail&continueMode=true&componentName=rec-play&originRequestUrl=https%3A%2F%2Fwse.zoom.us%2Frec%2Fshare%2FzQBgnS1_pMLwerninaxoo3x1CcRQo7HSnZDO_CDCD08-DWoBvBX5mLGFOeLmAcup.TLMB3hEbKsHZpKvd%3F_x_zm_rtaid=sWUOh7_xTSCQHcU845hLdA.1683232100781.2d589fddffcaf02d71517f5e6025816e&_x_zm_rhtaid=923
- Thank you Jordan for always grading homework in a timely manner and providing useful feedback. 
    - Plus your office hours were always very organized and helpful

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.